# 本ノートブックについて

作業内容
1. データベースから取得したい, xbrlのd情報を抽出
2. doc_idからEDINET APIを使用し，xbrlを取得する


In [2]:
from settings import *
import sqlite3
import urllib3
import certifi
import os
import requests
import time
from pathlib import Path
from urllib3.exceptions import InsecureRequestWarning
#InsecureRequestWarningを無視
urllib3.disable_warnings(InsecureRequestWarning)

In [4]:

def get_xbrl_dir_path() :
    """
    XBRLファイルの保存ディレクトリを取得する
    """
    
    return Path("../01_data/CA")

def get_xbrl_file_path(doc_id) :
    """
    XBRLファイルの保存パスを取得する
    """
    return get_xbrl_dir_path().joinpath(f"{doc_id}.zip")

def download_edinet_xbrl(doc_id) :
	"""
	EDINETからXBRLをダウンロードする
	Parameters
		doc_id : string
			EDINETのXBRLファイルのID
	Returns
		download_result : boolean
			ダウンロード成功したかどうか
	"""
    

	#ダウンロード結果
	download_result = False
	#リクエストパラメータ
	params = {
		'type' : 1
	}
 	#リクエストURL
	url = f'https://disclosure.edinet-fsa.go.jp/api/v1/documents/{doc_id}'
	
	#リクエスト送信
	#リトライ回数
	retry_count = -1
	#10回リトライする
	while retry_count < 10:
		retry_count = retry_count + 1

		print(url + 'へリクエスト送信')

		time.sleep(2.0)

		try:
			#リクエスト送信
			res = requests.get(url, \
					params=params, \
					verify=False, \
					timeout = 120.0)

		except requests.exceptions.RequestException as e:

			print('requests error' + str(e) )

			time.sleep(10.0)
			continue
		
		#ステータスコードが200以外の場合はリトライする
		if res.status_code != 200 :

			print('status_code:' + res.status_code)
			res.close()
			time.sleep(10.0)
			continue
		
		#content-typeがapplication/octet-streamでない場合はリトライする
		if res.headers['content-type'] != 'application/octet-stream' :

			print('content-type:' + res.headers['content-type'])

			res.close()
			time.sleep(10.0)
			continue

		#XBRLをローカルに保存する
		if not os.path.exists( get_xbrl_dir_path() ) :
			os.makedirs( get_xbrl_dir_path() )

		with open(get_xbrl_file_path(doc_id), 'wb') as f :
			for chunk in res.iter_content(chunk_size=1024) :
				f.write(chunk)

		res.close()


		download_result = True

		break


	return download_result

In [5]:
class EdinetDocumentInfoRecord() :
	"""
    EDINETの書類情報を保持するクラス
	"""


	def __init__(self, seq_number, \
				doc_id, \
				edinet_code, \
				sec_code, \
				jcn, \
				filer_name, \
				fund_code, \
				ordinance_code, \
				form_code, \
				doc_type_code, \
				period_start, \
				period_end, \
				submit_date_time, \
				doc_description, \
				issuer_edinet_code, \
				subject_edinet_code, \
				subsidiary_edinet_code, \
				current_report_reason, \
				parent_doc_id, \
				ope_date_time, \
				withdrawal_status, \
				doc_info_edit_status, \
				disclosure_status, \
				xbrl_flag, \
				pdf_flag, \
				attach_doc_flag, \
				english_doc_flag, \
				csv_flag, \
				legal_status, \
				json_date):


		self.__seq_number=seq_number
		self.__doc_id=doc_id
		self.__edinet_code=edinet_code
		self.__sec_code=sec_code
		self.__jcn=jcn
		self.__filer_name=filer_name
		self.__fund_code=fund_code
		self.__ordinance_code=ordinance_code
		self.__form_code=form_code
		self.__doc_type_code=doc_type_code
		self.__period_start=period_start
		self.__period_end=period_end
		self.__submit_date_time=submit_date_time
		self.__doc_description=doc_description
		self.__issuer_edinet_code=issuer_edinet_code
		self.__subject_edinet_code=subject_edinet_code
		self.__subsidiary_edinet_code=subsidiary_edinet_code
		self.__current_report_reason=current_report_reason
		self.__parent_doc_id=parent_doc_id
		self.__ope_date_time=ope_date_time
		self.__withdrawal_status=withdrawal_status
		self.__doc_info_edit_status=doc_info_edit_status
		self.__disclosure_status=disclosure_status
		self.__xbrl_flag=xbrl_flag
		self.__pdf_flag=pdf_flag
		self.__attach_doc_flag=attach_doc_flag
		self.__english_doc_flag=english_doc_flag
		self.__csv_flag=csv_flag
		self.__legal_status=legal_status
		self.__json_date=json_date

	def get_doc_id(self) :
		return self.__doc_id

	def get_edinet_code(self) :
		return self.__edinet_code

	def get_filer_name(self) :
		return self.__filer_name

	def get_doc_description(self) :
		return self.__doc_description


# データベースから取得したいxbrlの情報を取得する

参考資料
- doc_type_codeについて
  - 有価証券や大量保有報告書など種類を表すコード
  - 詳細は下記PDF参照
  - /Users/momoidaisuke/practice/edinet/00_ref/EDINET_API仕様書.pdf
- edinet_code
  - 企業のコード
  - /Users/momoidaisuke/practice/edinet/00_ref/EdinetcodeDlInfo.csv

In [6]:
#検索結果を保存する
edinet_document_info_record_list = list()

# dbに接続
conn = sqlite3.connect(DB_PATH)
# カラム名でアクセスできるようにする
conn.row_factory = sqlite3.Row

# カーソルを取得する
cur = conn.cursor()

# 検索
cur.execute('SELECT * FROM edinet_document \
		WHERE edinet_code=? and doc_type_code=? AND xbrl_flag=?',\
		 ('E05072', '120', '1'))

# 検索結果を一行ずつ取得する
row = cur.fetchone()
while row != None :
    # EdinetDocumentInfoRecordクラスを作成する
	search_result = EdinetDocumentInfoRecord( row['seq_number'],\
					row['doc_id'],\
					row['edinet_code'],\
					row['sec_code'],\
					row['jcn'],\
					row['filer_name'],\
					row['fund_code'],\
					row['ordinance_code'],\
					row['form_code'],\
					row['doc_type_code'],\
					row['period_start'],\
					row['period_end'],\
					row['submit_date_time'],\
					row['doc_description'],\
					row['issuer_edinet_code'],\
					row['subject_edinet_code'],\
					row['subsidiary_edinet_code'],\
					row['current_report_reason'],\
					row['parent_doc_id'],\
					row['ope_date_time'],\
					row['withdrawal_status'],\
					row['doc_info_edit_status'],\
					row['disclosure_status'],\
					row['xbrl_flag'],\
					row['pdf_flag'],\
					row['attach_doc_flag'],\
					row['english_doc_flag'],\
					row['csv_flag'],\
					row['legal_status'],\
					row['json_date'])


	edinet_document_info_record_list.append(search_result)

	row = cur.fetchone() 

# カーソルを閉じる
cur.close()
# dbを閉じる
conn.close()

In [7]:
print(len(edinet_document_info_record_list))

10


In [8]:
#XBRLをダウンロードする
#取得に失敗したファイルを保存する
fail_list = list()
for search_result in edinet_document_info_record_list :
	#検索結果のファイル名を表示する
	print(search_result.get_filer_name() + ',' + \
		search_result.get_doc_id() + ',' + \
		search_result.get_doc_description())

	#doc_idを取得する
	doc_id = search_result.get_doc_id()

	#XBRLファイルが存在するか確認する
	#存在する場合はスキップする
	if get_xbrl_file_path(doc_id).exists() :
		print('xbrl is already exists. pass')
		continue

	#XBRLファイルをダウンロードする
	result = download_edinet_xbrl(doc_id)
    #ダウンロードに失敗した場合はエラーメッセージを表示する
	if result == False :
		fail_list.append(doc_id)
		print(f'{doc_id}download fail')
		continue
    #ダウンロードに成功した場合はメッセージを表示する
	print(f'{doc_id}download success')

株式会社サイバーエージェント,S1000QUQ,有価証券報告書－第16期(2012/10/01－2013/09/30)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S1000QUQへリクエスト送信
S1000QUQdownload success
株式会社サイバーエージェント,S1003O7E,有価証券報告書－第17期(2013/10/01－2014/09/30)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S1003O7Eへリクエスト送信
S1003O7Edownload success
株式会社サイバーエージェント,S1006JW2,有価証券報告書－第18期(2014/10/01－2015/09/30)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S1006JW2へリクエスト送信
S1006JW2download success
株式会社サイバーエージェント,S1009BLV,有価証券報告書－第19期(2015/10/01－2016/09/30)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S1009BLVへリクエスト送信
S1009BLVdownload success
株式会社サイバーエージェント,S100C0N6,有価証券報告書－第20期(2016/10/01－2017/09/30)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100C0N6へリクエスト送信
S100C0N6download success
株式会社サイバーエージェント,S100ESN9,有価証券報告書－第21期(平成29年10月1日－平成30年9月30日)
https://disclosure.edinet-fsa.go.jp/api/v1/documents/S100ESN9へリクエスト送信
S100ESN9download success
株式会社サイバーエージェント,S100HMPO,有価証券報告書－第22期(平成30年10月1日－令和1年9月30日)
https://dis

MEMO: XBRLにはS100BHAPまでが含まれている